In [ ]:
import pandas as pd
import pickle
from scipy import stats
import pandas as pd
import seaborn as sns
import pingouin as pg
import statsmodels.formula.api as s
import statsmodels.api as sm
from statsmodels.formula.api import ols


In [ ]:
# load data
final = pickle.load(open( '../00_data/final/incorrect_answers.pkl','rb') )

# split by interventiongroup
intvcontrol = final[(final['interventiongroup']=='control')]
intv2 = final[(final['interventiongroup']=='2')]
intv3 = final[(final['interventiongroup']=='3')]
intv4 = final[(final['interventiongroup']=='4')]
intv5 = final[(final['interventiongroup']=='5')]
intv6 = final[(final['interventiongroup']=='6')]

Check assumptions

In [ ]:
# Levenes homogeneity 
stats.levene(intvcontrol['prop_fehler'], intv2['prop_fehler'], intv3['prop_fehler'], intv4['prop_fehler'], intv5['prop_fehler'], intv6['prop_fehler'])

In [ ]:
# Shapiro-Wilk test for normality 
print(stats.shapiro(intvcontrol['prop_fehler']))
print(stats.shapiro(intv2['prop_fehler']))
print(stats.shapiro(intv3['prop_fehler']))
print(stats.shapiro(intv4['prop_fehler']))
print(stats.shapiro(intv5['prop_fehler']))
print(stats.shapiro(intv6['prop_fehler']))

In [ ]:
# plot histogram
sns.histplot(data=final, x="prop_fehler")

Effect sizes

In [ ]:
# get effect size by interventiongroup
prop_fehler_by_group = final.groupby(['interventiongroup']).agg({'prop_fehler':'median','Geschlecht':'count'})
prop_fehler_by_group = prop_fehler_by_group.rename(columns={'Geschlecht':'count'})

In [ ]:
# get effect size by interventiongroup and gender
prop_fehler_by_group = final.groupby(['Geschlecht','interventiongroup']).agg({'prop_fehler':'median','Klassenstufe':'count'})
prop_fehler_by_group = prop_fehler_by_group.rename(columns={'Klassenstufe':'count'})

In [ ]:
# get effect size by interventiongroup and class level
prop_fehler_by_group = final.groupby(['Klassenstufe','interventiongroup']).agg({'prop_fehler':'median','Geschlecht':'count'})
prop_fehler_by_group = prop_fehler_by_group.rename(columns={'Geschlecht':'count'})

Check significance

In [ ]:
# Kruskal Wallis Test 
stats.kruskal(intvcontrol['prop_fehler'], intv2['prop_fehler'], intv3['prop_fehler'],intv4['prop_fehler'],intv5['prop_fehler'],intv6['prop_fehler'])

In [ ]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv2['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv3['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv4['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv5['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv6['prop_fehler']))

In [ ]:
# boxplot by interventiongroup
sns.boxplot(data=final, x="prop_fehler", y="interventiongroup",showfliers = False)#,hue='Geschlecht'


In [ ]:
# boxplot by interventiongroup and gender
sns.boxplot(data=final, x="prop_fehler", hue="interventiongroup",y='Geschlecht',showfliers = False)

Gender Demographics

In [ ]:
# split by gender
final_m = final.loc[final.Geschlecht == 'm']
final_w = final.loc[final.Geschlecht == 'w']
final_d = final.loc[final.Geschlecht == 'd']

Boys

In [ ]:
# split by interventiongroup
intvcontrol_m = final_m[(final_m['interventiongroup']=='control')]
intv2_m = final_m[(final_m['interventiongroup']=='2')]
intv3_m = final_m[(final_m['interventiongroup']=='3')]
intv4_m = final_m[(final_m['interventiongroup']=='4')]
intv5_m = final_m[(final_m['interventiongroup']=='5')]
intv6_m = final_m[(final_m['interventiongroup']=='6')]

# Kruskal Wallis Test 
stats.kruskal(intvcontrol_m['prop_fehler'], intv2_m['prop_fehler'], intv3_m['prop_fehler'],intv4_m['prop_fehler'],intv5_m['prop_fehler'],intv6_m['prop_fehler'])

In [ ]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(intvcontrol_m['prop_fehler'], intv2_m['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_m['prop_fehler'], intv3_m['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_m['prop_fehler'], intv4_m['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_m['prop_fehler'], intv5_m['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_m['prop_fehler'], intv6_m['prop_fehler']))

In [ ]:
# boxplot by prop fehler and interventiongroup
sns.boxplot(data=final_m, x="prop_fehler", y="interventiongroup",showfliers = False)

Girls

In [ ]:
# split by interventiongroup
intvcontrol_w = final_w[(final_w['interventiongroup']=='control')]
intv2_w = final_w[(final_w['interventiongroup']=='2')]
intv3_w = final_w[(final_w['interventiongroup']=='3')]
intv4_w = final_w[(final_w['interventiongroup']=='4')]
intv5_w = final_w[(final_w['interventiongroup']=='5')]
intv6_w = final_w[(final_w['interventiongroup']=='6')]

# Kruskal Wallis Test 
stats.kruskal(intvcontrol_w['prop_fehler'], intv2_w['prop_fehler'], intv3_w['prop_fehler'],intv4_w['prop_fehler'],intv5_w['prop_fehler'],intv6_w['prop_fehler'])


In [ ]:
#Wilcoxon Mann Whitney Test
print(stats.mannwhitneyu(intvcontrol_w['prop_fehler'], intv2_w['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_w['prop_fehler'], intv3_w['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_w['prop_fehler'], intv4_w['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_w['prop_fehler'], intv5_w['prop_fehler']))
print(stats.mannwhitneyu(intvcontrol_w['prop_fehler'], intv6_w['prop_fehler']))

In [ ]:
# boxplot prop fehler / interventiongroup
sns.boxplot(data=final_w, x="prop_fehler", y="interventiongroup",showfliers = False)

Divers

In [ ]:
# intvcontrol_d = final_d[(final_d['interventiongroup']=='control')]
# intv2_d = final_d[(final_d['interventiongroup']=='2')]
# intv3_d = final_d[(final_d['interventiongroup']=='3')]
# intv4_d = final_d[(final_d['interventiongroup']=='4')]
# intv5_d = final_d[(final_d['interventiongroup']=='5')]
# intv6_d = final_d[(final_d['interventiongroup']=='6')]

# # Kruskal Wallis Test 
# stats.kruskal(intvcontrol_d['prop_fehler'], intv2_d['prop_fehler'], intv3_d['prop_fehler'],intv4_d['prop_fehler'],intv5_d['prop_fehler'],intv6_d['prop_fehler'])

# #Wilcoxon Mann Whitney Test
# print(stats.mannwhitneyu(intvcontrol_d['prop_fehler'], intv2_d['prop_fehler']))
# print(stats.mannwhitneyu(intvcontrol_d['prop_fehler'], intv3_d['prop_fehler']))
# print(stats.mannwhitneyu(intvcontrol_d['prop_fehler'], intv4_d['prop_fehler']))
# print(stats.mannwhitneyu(intvcontrol_d['prop_fehler'], intv5_d['prop_fehler']))
# print(stats.mannwhitneyu(intvcontrol_d['prop_fehler'], intv6_d['prop_fehler']))

# gr = sns.boxplot(data=final_d, x="prop_fehler", y="interventiongroup",showfliers = False)
# gr.set(xlabel ="Incorrect answers", ylabel = "Intervention group")

Demographics Klassenstufe

In [ ]:
# split by class level
final_5 = final.loc[final.Klassenstufe == '5']
final_6 = final.loc[final.Klassenstufe == '6']
final_7 = final.loc[final.Klassenstufe == '7']
final_8 = final.loc[final.Klassenstufe == '8']
final_9 = final.loc[final.Klassenstufe == '9']
final_10 = final.loc[final.Klassenstufe == '10']
final_11 = final.loc[final.Klassenstufe == '11']
final_12 = final.loc[final.Klassenstufe == '12']
final_13 = final.loc[final.Klassenstufe == '13']

klassenliste = [final_5, final_6, final_7, final_8, final_9, final_10, final_11, final_12, final_13]

for x in klassenliste:
    intvcontrol = x[(x['interventiongroup']=='control')]
    intv2 = x[(x['interventiongroup']=='2')]
    intv3 = x[(x['interventiongroup']=='3')]
    intv4 = x[(x['interventiongroup']=='4')]
    intv5 = x[(x['interventiongroup']=='5')]
    intv6 = x[(x['interventiongroup']=='6')]

    # Kruskal Wallis Test 
    print(stats.kruskal(intvcontrol['prop_fehler'], intv2['prop_fehler'], intv3['prop_fehler'],intv4['prop_fehler'],intv5['prop_fehler'],intv6['prop_fehler']))


In [ ]:
# carry out wilcoxon mann whitney test for significant class levels

klassenliste = [final_5, final_6, final_7, final_8, final_9, final_10, final_11]

for x in klassenliste:
    intvcontrol = x[(x['interventiongroup']=='control')]
    intv2 = x[(x['interventiongroup']=='2')]
    intv3 = x[(x['interventiongroup']=='3')]
    intv4 = x[(x['interventiongroup']=='4')]
    intv5 = x[(x['interventiongroup']=='5')]
    intv6 = x[(x['interventiongroup']=='6')]

    #Wilcoxon Mann Whitney Test
    print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv2['prop_fehler']))
    print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv3['prop_fehler']))
    print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv4['prop_fehler']))
    print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv5['prop_fehler']))
    print(stats.mannwhitneyu(intvcontrol['prop_fehler'], intv6['prop_fehler']))
    print('--------------')

In [ ]:
# Boxplot 

In [ ]:
# print(sns.boxplot(data=final_5, x="prop_fehler", y="interventiongroup",showfliers = False))

In [ ]:
# print(sns.boxplot(data=final_8, x="prop_fehler", y="interventiongroup",showfliers = False))

In [ ]:
# print(sns.boxplot(data=final_9, x="prop_fehler", y="interventiongroup",showfliers = False))